In [2]:
import numpy as np


In [39]:
# This is a code to simulate the interaction between two electron beams in a 1D plasma using an electrostatic PIC code, the length is measured in Debye length
length_of_system = 64

# dt or time_step represents the temporal time step in units of plasma period
dt = 0.1

#Frequencies are measured in units of plasma frequency 
plasma_frequency = 1;

# iteration number
iterations_number = 10

# final time

final_time = (iterations_number)*dt

# number of grid points
grid_points = 256
number_of_beams =  2

# Beam 1
number_of_particles_in_beam1= 10000
drift_velocity_beam1 = 5 #Velocities are measured in units of thermal electron velocity
thermal_velocity_beam1 = 1
charge_mass_relation_beam1 = -1;
perturbation_amplitude_beam_1 = 0;
mode_beam1 = 0;


# Beam 2
number_of_particles_in_beam2= 10000
drift_velocity_beam2 = -5 #Velocities are measured in units of thermal electron velocity
thermal_velocity_beam2 = 1
charge_mass_relation_beam2 = -1; 
perturbation_amplitude_beam_2 = 0;
mode_beam2 = 0;


#Ions in the background
number_of_background_ions = 20000;

#Size of the cell
dx = length_of_system/grid_points;

time=np.arange((iterations_number+1)*dt,dt)




In [14]:
def charge(
    charge_mass_beam1: float,
    charge_mass_beam2: float,
    number_of_background_ions: int,
    number_of_particles_in_beam1: int,
    number_of_particles_in_beam2: int,
    length_of_system: float,
    plasma_frequency: float
    ):
    q_1 = (plasma_frequency**2 * length_of_system) / (number_of_particles_in_beam1 * charge_mass_beam1)
    if charge_mass_beam2 > 0:
        charge_mass_beam2 = -q_1 * (number_of_particles_in_beam1 / number_of_particles_in_beam2)
    elif charge_mass_beam2 < 0:
        q_2 = q_1 * (number_of_particles_in_beam1 / number_of_particles_in_beam2)
    else:
        q_2 = 0
    rho_back = (-q_1 / length_of_system) * (number_of_background_ions)
    return q_1, q_2, rho_back

In [40]:
import numpy as np
import matplotlib.pyplot as plt

# Parameters
length_of_system = 64
dt = 0.1
plasma_frequency = 1
iterations_number = 10
final_time = iterations_number * dt
grid_points = 256
number_of_beams = 2

# Beam 1
number_of_particles_in_beam1 = 10000
drift_velocity_beam1 = 5
thermal_velocity_beam1 = 1
charge_mass_relation_beam1 = -1
perturbation_amplitude_beam_1 = 0
mode_beam1 = 0

# Beam 2
number_of_particles_in_beam2 = 10000
drift_velocity_beam2 = -5
thermal_velocity_beam2 = 1
charge_mass_relation_beam2 = -1
perturbation_amplitude_beam_2 = 0
mode_beam2 = 0

# Ions in the background
number_of_background_ions = 20000

# Size of the cell
dx = length_of_system / grid_points
time = np.arange(0, final_time + dt, dt)

# Charge
def Charge(QM1, QM2, IB, N1, N2, L, WP):
    Q1 = QM1 * N1
    Q2 = QM2 * N2
    rho_back = IB / L
    return Q1, Q2, rho_back

Q1, Q2, rho_back = Charge(charge_mass_relation_beam1, charge_mass_relation_beam2, number_of_background_ions, number_of_particles_in_beam1, number_of_particles_in_beam2, length_of_system, 1)

# Initial loading
def InitialLoading(N, V0, Vth, XP, Mode, L):
    xp = np.random.uniform(0, L, N)
    vp = np.random.normal(V0, Vth, N)
    return xp, vp

xp1, vp1 = InitialLoading(number_of_particles_in_beam1, drift_velocity_beam1, thermal_velocity_beam1, perturbation_amplitude_beam_1, mode_beam1, length_of_system)
xp2, vp2 = InitialLoading(number_of_particles_in_beam2, drift_velocity_beam2, thermal_velocity_beam2, perturbation_amplitude_beam_2, mode_beam2, length_of_system)

# Auxiliarity vectors
def AuxVector(N):
    return np.arange(N)

p1 = AuxVector(number_of_particles_in_beam1)
p2 = AuxVector(number_of_particles_in_beam2)

# Poisson's equation preparation
un = np.ones(grid_points - 1)
Ax = np.diag(un, -1) - 2 * np.diag(un) + np.diag(un, 1)
kap = (2 * np.pi / length_of_system) * np.arange(-grid_points / 2, grid_points / 2)
kap = np.fft.fftshift(kap)
kap[0] = 1

# Others
mat1 = 0
mat2 = 0
Eg = 0
Phi = 0

# Computational cycle
mom = np.zeros(iterations_number + 1)
E_kin = np.zeros(iterations_number + 1)
E_pot = np.zeros(iterations_number + 1)

for it in range(iterations_number + 1):
    mom[it] = (Q1 / charge_mass_relation_beam1) * np.sum(vp1) + (Q2 / charge_mass_relation_beam2) * np.sum(vp2)
    E_kin[it] = 0.5 * np.abs(Q1 / charge_mass_relation_beam1) * np.sum(vp1 ** 2) + 0.5 * np.abs(Q2 / charge_mass_relation_beam2) * np.sum(vp2 ** 2)
    E_pot[it] = 0.5 * np.sum(Eg ** 2) * dx

    # Updating velocity
    vp1 = MotionV(vp1, charge_mass_relation_beam1, mat1, Eg, number_of_particles_in_beam1, 'Leapfrog', dt)
    vp2 = MotionV(vp2, charge_mass_relation_beam2, mat2, Eg, number_of_particles_in_beam2, 'Leapfrog', dt)

    # Updating positions
    xp1 = MotionX(xp1, vp1, 'Leapfrog', dt)
    xp2 = MotionX(xp2, vp2, 'Leapfrog', dt)

    # Periodic boundary conditions:
    xp1 = PeriodicBC(xp1, 0, length_of_system)
    xp2 = PeriodicBC(xp2, 0, length_of_system)

    # Interpolation functions
    mat1 = InterpolationF('Cloud in Cell (CIC)', dx, grid_points, xp1, number_of_particles_in_beam1, p1)
    mat2 = InterpolationF('Cloud in Cell (CIC)', dx, grid_points, xp2, number_of_particles_in_beam2, p2)

    # Charge density:
    rho1 = Charge_density(Q1, mat1, dx)
    rho2 = Charge_density(Q2, mat2, dx)
    rhot = rho1 + rho2 + rho_back

    # Field equations:
    Phi, Eg = Field('Finite Difference Method', rhot, grid_points, dx, length_of_system, Ax, kap)

    # Updating velocity
    vp1 = MotionV(vp1, charge_mass_relation_beam1, mat1, Eg, number_of_particles_in_beam1, 'Leapfrog', dt)
    vp2 = MotionV(vp2, charge_mass_relation_beam2, mat2, Eg, number_of_particles_in_beam2, 'Leapfrog', dt)

    plt.scatter(xp1, vp1)
    plt.hold(True)
    plt.scatter(xp2, vp2)
    plt.pause(0.00001)
    plt.hold(False)

plt.figure()
plt.plot(time, mom)
plt.xlabel('Time')
plt.ylabel('Momentum')

plt.figure()
plt.plot(time, E_kin, 'k')
plt.plot(time, E_pot, 'r')
plt.plot(time, E_kin + E_pot, 'b')
plt.xlabel('Time')
plt.ylabel('Energy')

ValueError: operands could not be broadcast together with shapes (256,256) (255,255) 